1\. Write a function that converts number representation (bin<->dec<->hex)

In [ ]:
def first2(s):
    return s[:2]

def converter(a):
    if type(a) == int:
        print("your number in binary = ", bin(a))
        print("your number in hexadecimal = ", hex(a))
    elif first2(a) == "0x":
        print("your number in decimal = ", int(a,16))
        print("your number in binary = ", bin(int(a,16)))
    elif first2(a) == "0b":
        print("your number in decimal = ", int(a,2))
        print("your number in hexadecimal = ", hex(int(a,2)))

#Write your binary and hexadecimal number as a string with the prefix "0b" or "0x"
converter("0x2a")


2\. Write a function that converts a 32 bit word into a single precision floating point (i.e. interprets the various bits as sign, mantissa and exponent)

3\. Write a program to determine the underflow and overflow limits (within a factor of 2) for python on your computer. 

**Tips**: define two variables inizialized to 1 and halve/double them enough time to exceed the under/over-flow limits  

In [2]:
import math

over = 1.
under = 1.

while((under/2)>0):
    under = under/2
print("Underflow limit: ", under)

while((over*2)<math.inf):
    over = over*2
print("Overflow limit: ", over)

Underflow limit:  5e-324
Overflow limit:  8.98846567431158e+307


4\. Write a program to determine the machine precision

**Tips**: define a new variable by adding a smaller and smaller value (proceeding similarly to prob. 2) to an original variable and check the point where the two are the same 

In [4]:
a = 1
b = 1

while(b==1):
    a = a/2  
    if (a+b == b):
        b = 0

print(a)

1.1102230246251565e-16


5\. Write a function that takes in input three parameters $a$, $b$ and $c$ and prints out the two solutions to the quadratic equation $ax^2+bx+c=0$ using the standard formula:
$$
x=\frac{-b\pm\sqrt{b^2-4ac}}{2a}
$$

(a) use the program to compute the solution for $a=0.001$, $b=1000$ and $c=0.001$

(b) re-express the standard solution formula by multiplying top and bottom by $-b\mp\sqrt{b^2-4ac}$ and again find the solution for $a=0.001$, $b=1000$ and $c=0.001$. How does it compare with what previously obtained? Why?

(c) write a function that compute the roots of a quadratic equation accurately in all cases

In [21]:
from math import sqrt

a = 0.001
b = 1000
c = 0.001

def sol_1(a,b,c):
    x_1 = (-b + sqrt(b**2 -4*a*c))/(2*a)
    x_2 = (-b - sqrt(b**2 -4*a*c))/(2*a) 
    return [x_1, x_2]

print("solution 1:", sol_1(a,b,c))

def sol_2(a,b,c):    
    x_1 = (2*a)/(-b - sqrt(b**2 -4*a*c)) 
    x_2 = (-2*a)/(-b + sqrt(b**2 -4*a*c))
    return [x_1, x_2]
print("solution 2:", sol_2(a,b,c))

#the first solution have some problems with x_1 because the ratio (-b + sqrt(b**2 -4*a*c))/(2*a) is between 
#two small numbers, so the best solution in this case is sol_2.
#with the same argoument we can say that for x_2 the best solution is given by sol_1

#the optimal solution is therefore:

def sol_3(a,b,c):    
    x_1 = (2*a)/(-b - sqrt(b**2 -4*a*c)) 
    x_2 = (-b - sqrt(b**2 -4*a*c))/(2*a)
    return [x_1, x_2]
print("solution 3:", sol_3(a,b,c))


solution 1: [-9.999894245993346e-07, -999999.999999]
solution 2: [-1.000000000001e-06, 1000010.5755125057]
solution 3: [-1.000000000001e-06, -999999.999999]


6\. Write a program that implements the function $f(x)=x(x−1)$

(a) Calculate the derivative of the function at the point $x = 1$ using the derivative definition:

$$
\frac{{\rm d}f}{{\rm d}x} = \lim_{\delta\to0} \frac{f(x+\delta)-f(x)}{\delta}
$$

with $\delta = 10^{−2}$. Calculate the true value of the same derivative analytically and compare with the answer your program gives. The two will not agree perfectly. Why not?

(b) Repeat the calculation for $\delta = 10^{−4}, 10^{−6}, 10^{−8}, 10^{−10}, 10^{−12}$ and $10^{−14}$. How does the accuracy scales with $\delta$?

In [ ]:
delta = [10**(-x) for x in range(15) if x%2 == 0 and x!=0]
def f(x):
    return x*(x-1)

def derivative(x, d):
    return (f(x+d) - f(x))/d

for d in delta:
    print(derivative(1,d))
#with smaller delta we have a better accuracy, but with the last 2:  10^12,1 0^114  we lose accuracy

with smaller delta we have a better accuracy, but with the last 2: $10^{−12}, 10^{−14}$ we lose accuracy

7\. Consider the integral of the semicircle of radius 1:
$$
I=\int_{-1}^{1} \sqrt(1-x^2) {\rm d}x
$$
which it's known to be $I=\frac{\pi}{2}=1.57079632679...$.
Alternatively we can use the Riemann definition of the integral:
$$
I=\lim_{N\to\infty} \sum_{k=1}^{N} h y_k 
$$

with $h=2/N$ the width of each of the $N$ slices the domain is divided into and where
$y_k$ is the value of the function at the $k-$th slice.

(a) Write a programe to compute the integral with $N=100$. How does the result compares to the true value?

(b) How much can $N$ be increased if the computation needs to be run in less than a second? What is the gain in running it for 1 minute? 


In [ ]:
from math import sqrt
def y(x):
    return sqrt(1-x*x)

def integral(N):
    h = 2/N
    a, b = -1, 1
    n = 0
    I = 0.0
    x = a
    while n < N:
        delta_I = y(x) * h
        x = x + h
        I = I + delta_I
        n += 1
    return I

n = [10**(x) for x in range(8) if x >= 2]
for N in n:
    print(integral(N))
    %timeit integral(N)
    print()
    
#we reach 1 secondof runtime about between N = 100000 and N = 1000000
#so let's try with N = 4000000

print(integral(4000000))
%timeit integral(4000000)